# Text Summarization
Text summarization is a challenge to deal with huge size of texts that come with various forms, such as documents, emails, and web pages.

TextRank

In [5]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import contractions
import zipfile

In [4]:
pd.set_option("display.max_colwidth",1000)

The processes of opening the zip file

In [6]:
glove_dir ='data_ch6/glove/'
glove_zip = glove_dir + 'glove.6B.50d.zip'

In [7]:
zip_ref=zipfile.ZipFile(glove_zip,'r')
zip_ref.extractall(glove_dir)
zip_ref.close()

In [8]:
def load_glove_vectors(fn):
    print('Loading glove Model')
    with open(fn, 'r', encoding='utf8')as glove_vector_file:
        model={}
        for line in glove_vector_file:
            parts=line.split()
            word=parts[0]
            embedding=np.array([float(val) for val in parts[1:]])
            model[word]=embedding
        print('Loaded {} words'.format(len(model)))
    return model

In [9]:
glove_vectors=load_glove_vectors('data_ch6/glove/glove.6B.50d.txt')

Loading glove Model
Loaded 400000 words


In [10]:
doc=pd.read_csv('data_ch6/tennis_articles_v4.csv')

In [12]:
doc.head()

,article_id,article_text,source
0,1,"Maria Sharapova has basically no friends as tennis players on the WTA Tour. The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much. I think everyone knows this is my job here. When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match. I'm a pretty competitive girl. I say my hellos, but I'm not sending any players flowers as well. Uhm, I'm not really friendly or close to many players. I have not a lot of friends away from the courts.' When she said she is not really close to a lot of players, is that something strategic that she is doing? Is it different on the men's tour than the women's tour? 'No, not at all. I think just because you're in the sa...",https://www.tennisworldusa.org/tennis/news/Maria_Sharapova/62220/i-do-not-have-friends-in-tennis-says-maria-sharapova/
1,2,"BASEL, Switzerland (AP), Roger Federer advanced to the 14th Swiss Indoors final of his career by beating seventh-seeded Daniil Medvedev 6-1, 6-4 on Saturday. Seeking a ninth title at his hometown event, and a 99th overall, Federer will play 93th-ranked Marius Copil on Sunday. Federer dominated the 20th-ranked Medvedev and had his first match-point chance to break serve again at 5-1. He then dropped his serve to love, and let another match point slip in Medvedev's next service game by netting a backhand. He clinched on his fourth chance when Medvedev netted from the baseline. Copil upset expectations of a Federer final against Alexander Zverev in a 6-3, 6-7 (6), 6-4 win over the fifth-ranked German in the earlier semifinal. The Romanian aims for a first title after arriving at Basel without a career win over a top-10 opponent. Copil has two after also beating No. 6 Marin Cilic in the second round. Copil fired 26 aces past Zverev and never dropped serve, clinching after 2 1/2 hours w...",http://www.tennis.com/pro-game/2018/10/copil-stuns-5th-ranked-zverev-to-reach-swiss-indoors-final/77721/
2,3,"Roger Federer has revealed that organisers of the re-launched and condensed Davis Cup gave him three days to decide if he would commit to the controversial competition. Speaking at the Swiss Indoors tournament where he will play in Sundays final against Romanian qualifier Marius Copil, the world number three said that given the impossibly short time frame to make a decision, he opted out of any commitment. ""They only left me three days to decide"", Federer said. ""I didn't to have time to consult with all the people I had to consult. ""I could not make a decision in that time, so I told them to do what they wanted."" The 20-time Grand Slam champion has voiced doubts about the wisdom of the one-week format to be introduced by organisers Kosmos, who have promised the International Tennis Federation up to $3 billion in prize money over the next quarter-century. The competition is set to feature 18 countries in the November 18-24 finals in Madrid next year, and will replace the classic hom...",https://scroll.in/field/899938/tennis-roger-federer-ignored-deadline-set-by-new-davis-cup
3,4,"Kei Nishikori will try to end his long losing streak in ATP finals and Kevin Anderson will go for his second title of the year at the Erste Bank Open on Sunday. The fifth-seeded Nishikori reached his third final of 2018 after beating Mikhail Kukushkin of Kazakhstan 6-4, 6-3 in the semifinals. A winner of 11 ATP events, Nishikori hasn't triumphed since winning in Memphis in February 2016. He has lost eight straight finals since. The second-seeded Anderson defeated Fernando Verdasco 6-3, 3-6, 6-4. Anderson has a shot at a fifth career title and second of the year after winning in New York in February. Nishikori leads And

In [13]:
stop_words=stopwords.words("english")

Functions will clean the article text, as well as tokenizing into words

In [14]:
clean_pattern=r'[^a-zA-z\s]'
def clean(word):
    return re.sub(clean_pattern,'',word)

def clean_sentence(sent):
    sent=[clean(word) for word in sent]
    return [word for word in sent if word]

def clean_sentences(sents):
    return [clean_sentence(sent) for sent in sents ]

def lower(sen):
    return [w.lower() for w in sen]

def remove_stopwords(sw):
    words=[word for word in sw if word not in stop_words]
    return [word for word in words if len(word)>0]

def tokenize_words(token):
    return [word_tokenize(s) for s in token]

def fix_contractions(cont):
    return [contractions.fix(s) for s in cont]

Apply functions

In [16]:
doc['sentenceInArticle']=doc['article_text'].apply(sent_tokenize)
doc['WordInSentences']=doc['sentenceInArticle'].apply(fix_contractions)\
                                                .apply(lower).apply(tokenize_words)\
                                                .apply(remove_stopwords).apply(clean_sentences)

In [17]:
doc.head()

,article_id,article_text,source,sentenceInArticle,WordInSentences
0,1,"Maria Sharapova has basically no friends as tennis players on the WTA Tour. The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much. I think everyone knows this is my job here. When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match. I'm a pretty competitive girl. I say my hellos, but I'm not sending any players flowers as well. Uhm, I'm not really friendly or close to many players. I have not a lot of friends away from the courts.' When she said she is not really close to a lot of players, is that something strategic that she is doing? Is it different on the men's tour than the women's tour? 'No, not at all. I think just because you're in the sa...",https://www.tennisworldusa.org/tennis/news/Maria_Sharapova/62220/i-do-not-have-friends-in-tennis-says-maria-sharapova/,"[Maria Sharapova has basically no friends as tennis players on the WTA Tour., The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much., I think everyone knows this is my job here., When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match., I'm a pretty competitive girl., I say my hellos, but I'm not sending any players flowers as well., Uhm, I'm not really friendly or close to many players., I have not a lot of friends away from the courts.', When she said she is not really close to a lot of players, is that something strategic that she is doing?, Is it different on the men's tour than the women's tour?, 'No, not at all., I think just because you'...","[[maria, sharapova, has, basically, no, friends, as, tennis, players, on, the, wta, tour], [the, russian, player, has, no, problems, in, openly, speaking, about, it, and, in, a, recent, interview, she, said, i, do, not, really, hide, any, feelings, too, much], [i, think, everyone, knows, this, is, my, job, here], [when, i, am, on, the, courts, or, when, i, am, on, the, court, playing, i, am, a, competitor, and, i, want, to, beat, every, single, person, whether, they, are, in, the, locker, room, or, across, the, netso, i, am, not, the, one, to, strike, up, a, conversation, about, the, weather, and, know, that, in, the, next, few, minutes, i, have, to, go, and, try, to, win, a, tennis, match], [i, am, a, pretty, competitive, girl], [i, say, my, hellos, but, i, am, not, sending, any, players, flowers, as, well], [uhm, i, am, not, really, friendly, or, close, to, many, players], [i, have, not, a, lot, of, friends, away, from, the, courts], [when, she, said, she, is, not, really, close,..."
1,2,"BASEL, Switzerland (AP), Roger Federer advanced to the 14th Swiss Indoors final of his career by beating seventh-seeded Daniil Medvedev 6-1, 6-4 on Saturday. Seeking a ninth title at his hometown event, and a 99th overall, Federer will play 93th-ranked Marius Copil on Sunday. Federer dominated the 20th-ranked Medvedev and had his first match-point chance to break serve again at 5-1. He then dropped his serve to love, and let another match point slip in Medvedev's next service game by netting a backhand. He clinched on his fourth chance when Medvedev netted from the baseline. Copil upset expectations of a Federer final against Alexander Zverev in a 6-3, 6-7 (6), 6-4 win over the fifth-ranked German in the earlier semifinal. The Romanian aims for a first title after arriving at Basel without

Subset the articles

In [19]:
articles=doc[['sentenceInArticle','WordInSentences']]

In [21]:
articles.head()

,sentenceInArticle,WordInSentences
0,"[Maria Sharapova has basically no friends as tennis players on the WTA Tour., The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much., I think everyone knows this is my job here., When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match., I'm a pretty competitive girl., I say my hellos, but I'm not sending any players flowers as well., Uhm, I'm not really friendly or close to many players., I have not a lot of friends away from the courts.', When she said she is not really close to a lot of players, is that something strategic that she is doing?, Is it different on the men's tour than the women's tour?, 'No, not at all., I think just because you'...","[[maria, sharapova, has, basically, no, friends, as, tennis, players, on, the, wta, tour], [the, russian, player, has, no, problems, in, openly, speaking, about, it, and, in, a, recent, interview, she, said, i, do, not, really, hide, any, feelings, too, much], [i, think, everyone, knows, this, is, my, job, here], [when, i, am, on, the, courts, or, when, i, am, on, the, court, playing, i, am, a, competitor, and, i, want, to, beat, every, single, person, whether, they, are, in, the, locker, room, or, across, the, netso, i, am, not, the, one, to, strike, up, a, conversation, about, the, weather, and, know, that, in, the, next, few, minutes, i, have, to, go, and, try, to, win, a, tennis, match], [i, am, a, pretty, competitive, girl], [i, say, my, hellos, but, i, am, not, sending, any, players, flowers, as, well], [uhm, i, am, not, really, friendly, or, close, to, many, players], [i, have, not, a, lot, of, friends, away, from, the, courts], [when, she, said, she, is, not, really, close,..."
1,"[BASEL, Switzerland (AP), Roger Federer advanced to the 14th Swiss Indoors final of his career by beating seventh-seeded Daniil Medvedev 6-1, 6-4 on Saturday., Seeking a ninth title at his hometown event, and a 99th overall, Federer will play 93th-ranked Marius Copil on Sunday., Federer dominated the 20th-ranked Medvedev and had his first match-point chance to break serve again at 5-1., He then dropped his serve to love, and let another match point slip in Medvedev's next service game by netting a backhand., He clinched on his fourth chance when Medvedev netted from the baseline., Copil upset expectations of a Federer final against Alexander Zverev in a 6-3, 6-7 (6), 6-4 win over the fifth-ranked German in the earlier semifinal., The Romanian aims for a first title after arriving at Basel without a career win over a top-10 opponent., Copil has two after also beating No., 6 Marin Cilic in the second round., Copil fired 26 aces past Zverev and never dropped serve, clinching after 2 1...","[[basel, switzerland, ap, roger, federer, advanced, to, the, th, swiss, indoors, final, of, his, career, by, beating, seventhseeded, daniil, medvedev, on, saturday], [seeking, a, ninth, title, at, his, hometown, event, and, a, th, overall, federer, will, play, thranked, marius, copil, on, sunday], [federer, dominated, the, thranked, medvedev, and, had, his, first, matchpoint, chance, to, break, serve, again, at], [he, then, dropped, his, serve, to, love, and, let, another, match, point, slip, in, medvedev, s, next, service, game, by, netting, a, backhand], [he, clinched, on, his, fourth, chance, when, medvedev, netted, from, the, baseline], [copil, upset, expectations, of, a, federer, final, against, alexander, zverev, in, a, win, over, the, fifthranked, german, in, the, earlier, semifinal], [the, romanian, aims, for, a, first, title, after, arriving, at, basel, without, a, career, win, over, a, top, opponent], [copil, has, two, after, also

Create a sentence vectors

In [23]:
vector_size=50
empty_vector=np.zeros(vector_size)

In [24]:
def sentence_vector(sent):
    return sum([glove_vectors.get(word, empty_vector) for word in sent])/len(sent)

def sentences_to_vectors(sentences):
    return [sentence_vector(s) for s in sentences]

In [25]:
articles['SentenceVector']=articles['WordInSentences'].apply(sentences_to_vectors)

C:\Users\Raymond\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Create a similiarity matrix captures the degree to which one sentence is similiart to another

In [26]:
from sklearn.metrics.pairwise import cosine_similarity

In [27]:
def similarity_matrix(sent_vectors):
    sim_mat=np.zeros([len(sent_vectors), len(sent_vectors)])
    for i in range(len(sent_vectors)):
        for j in range(len(sent_vectors)):
            elment_i = sent_vectors[i].reshape(1, vector_size)
            elment_j = sent_vectors[j].reshape(1, vector_size)
            sim_mat[i][j] = cosine_similarity(elment_i,elment_j)[0,0]
            
            return sim_mat

In [30]:
articles['SimMatrix']=articles['SimMatrix']=articles['SentenceVector'].apply(similarity_matrix)

C:\Users\Raymond\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [37]:
articles

,sentenceInArticle,WordInSentences,SentenceVector,SimMatrix
0,"[Maria Sharapova has basically no friends as tennis players on the WTA Tour., The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much., I think everyone knows this is my job here., When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match., I'm a pretty competitive girl., I say my hellos, but I'm not sending any players flowers as well., Uhm, I'm not really friendly or close to many players., I have not a lot of friends away from the courts.', When she said she is not really close to a lot of players, is that something strategic that she is doing?, Is it different on the men's tour than the women's tour?, 'No, not at all., I think just because you'...","[[maria, sharapova, has, basically, no, friends, as, tennis, players, on, the, wta, tour], [the, russian, player, has, no, problems, in, openly, speaking, about, it, and, in, a, recent, interview, she, said, i, do, not, really, hide, any, feelings, too, much], [i, think, everyone, knows, this, is, my, job, here], [when, i, am, on, the, courts, or, when, i, am, on, the, court, playing, i, am, a, competitor, and, i, want, to, beat, every, single, person, whether, they, are, in, the, locker, room, or, across, the, netso, i, am, not, the, one, to, strike, up, a, conversation, about, the, weather, and, know, that, in, the, next, few, minutes, i, have, to, go, and, try, to, win, a, tennis, match], [i, am, a, pretty, competitive, girl], [i, say, my, hellos, but, i, am, not, sending, any, players, flowers, as, well], [uhm, i, am, not, really, friendly, or, close, to, many, players], [i, have, not, a, lot, of, friends, away, from, the, courts], [when, she, said, she, is, not, really, close,...","[[0.07161230769230773, 0.5263353076923079, -0.22625407692307692, 0.4190584615384615, -0.0952153076923077, -0.10156600000000002, -0.40362538461538466, 0.400633476923077, -0.35831371000000006, -0.11500546153846154, 0.11607930769230769, -0.146259, -0.6847891923076924, 0.014327538461538456, 0.783284653846154, 0.046932615384615385, -0.10918245384615388, 0.054596461538461545, -0.29416438461538463, -0.1929796923076923, -0.2468181230769231, 0.3106658461538461, 0.011112153846153848, 0.30192715384615376, 0.12148384615384614, -1.328529230769231, -0.2920881538461539, -0.07721707692307692, -0.3258306923076923, -0.36910523076923074, 2.7283676923076925, 0.24652538461538465, -0.041018076923076914, 0.07203453846153846, 0.022849225384615382, -0.01111693076923078, 0.2808099230769231, 0.4716086153846154, -0.020941546153846143, -0.6126247692307694, -0.007031076923076909, -0.08967269230769231, 0.18087161538461535, 0.06510876923076923, 0.1128052307692308, -0.027662769230769227, -0.06883483846153847, -0.3...","[[1.0000000000000002, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

Next step is to create similarity matrices with graph from matrix.

In [31]:
import networkx as nx

In [38]:
def matrix_graph(sim_mat):
    nx_graph=nx.from_numpy_array(sim_mat)
    scores=nx.pagerank(nx_graph)
    return scores

In [39]:
articles['graph']=articles['SimMatrix'].apply(matrix_graph)

In [40]:
articles.head()

,sentenceInArticle,WordInSentences,SentenceVector,SimMatrix,graph
0,"[Maria Sharapova has basically no friends as tennis players on the WTA Tour., The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much., I think everyone knows this is my job here., When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match., I'm a pretty competitive girl., I say my hellos, but I'm not sending any players flowers as well., Uhm, I'm not really friendly or close to many players., I have not a lot of friends away from the courts.', When she said she is not really close to a lot of players, is that something strategic that she is doing?, Is it different on the men's tour than the women's tour?, 'No, not at all., I think just because you'...","[[maria, sharapova, has, basically, no, friends, as, tennis, players, on, the, wta, tour], [the, russian, player, has, no, problems, in, openly, speaking, about, it, and, in, a, recent, interview, she, said, i, do, not, really, hide, any, feelings, too, much], [i, think, everyone, knows, this, is, my, job, here], [when, i, am, on, the, courts, or, when, i, am, on, the, court, playing, i, am, a, competitor, and, i, want, to, beat, every, single, person, whether, they, are, in, the, locker, room, or, across, the, netso, i, am, not, the, one, to, strike, up, a, conversation, about, the, weather, and, know, that, in, the, next, few, minutes, i, have, to, go, and, try, to, win, a, tennis, match], [i, am, a, pretty, competitive, girl], [i, say, my, hellos, but, i, am, not, sending, any, players, flowers, as, well], [uhm, i, am, not, really, friendly, or, close, to, many, players], [i, have, not, a, lot, of, friends, away, from, the, courts], [when, she, said, she, is, not, really, close,...","[[0.07161230769230773, 0.5263353076923079, -0.22625407692307692, 0.4190584615384615, -0.0952153076923077, -0.10156600000000002, -0.40362538461538466, 0.400633476923077, -0.35831371000000006, -0.11500546153846154, 0.11607930769230769, -0.146259, -0.6847891923076924, 0.014327538461538456, 0.783284653846154, 0.046932615384615385, -0.10918245384615388, 0.054596461538461545, -0.29416438461538463, -0.1929796923076923, -0.2468181230769231, 0.3106658461538461, 0.011112153846153848, 0.30192715384615376, 0.12148384615384614, -1.328529230769231, -0.2920881538461539, -0.07721707692307692, -0.3258306923076923, -0.36910523076923074, 2.7283676923076925, 0.24652538461538465, -0.041018076923076914, 0.07203453846153846, 0.022849225384615382, -0.01111693076923078, 0.2808099230769231, 0.4716086153846154, -0.020941546153846143, -0.6126247692307694, -0.007031076923076909, -0.08967269230769231, 0.18087161538461535, 0.06510876923076923, 0.1128052307692308, -0.027662769230769227, -0.06883483846153847, -0.3...","[[1.0000000000000002, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

The graph contains a score and numeric index to a sentence
Next create a func that will rank the scores and return the top n sentencesby graph scores

In [43]:
def get_ranks(sent, scores, n=3):
    top_scores=sorted(((scores[i], s) for i,s in enumerate(sent)), reverse=True)
    top_n_sent=[s for score,s in top_scores[:n] ]
    
    return ' '.join(top_n_sent)

In [45]:
articles['summary']=articles.apply(lambda d: get_ranks(d.sentenceInArticle, d.graph),axis=1)

In [46]:
# summary article
articles.loc[0].summary

"Maria Sharapova has basically no friends as tennis players on the WTA Tour. When she said she is not really close to a lot of players, is that something strategic that she is doing? When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match."

In [48]:
# compare to original article
articles.loc[0].sentenceInArticle

['Maria Sharapova has basically no friends as tennis players on the WTA Tour.',
 "The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much.",
 'I think everyone knows this is my job here.',
 "When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match.",
 "I'm a pretty competitive girl.",
 "I say my hellos, but I'm not sending any players flowers as well.",
 "Uhm, I'm not really friendly or close to many players.",
 "I have not a lot of friends away from the courts.'",
 'When she said she is not really close to a lot of players, is that something strategic that she is doing?',
 "Is it different on the men's tour than the women's tour?",
 "'No, not at all.

In [49]:
# summary article
articles.loc[2].summary

'Roger Federer has revealed that organisers of the re-launched and condensed Davis Cup gave him three days to decide if he would commit to the controversial competition. We will see what happens," he said. This was designed for the future generation of players."'

In [50]:
# summary article
# compare to original article
articles.loc[2].sentenceInArticle

['Roger Federer has revealed that organisers of the re-launched and condensed Davis Cup gave him three days to decide if he would commit to the controversial competition.',
 'Speaking at the Swiss Indoors tournament where he will play in Sundays final against Romanian qualifier Marius Copil, the world number three said that given the impossibly short time frame to make a decision, he opted out of any commitment.',
 '"They only left me three days to decide", Federer said.',
 '"I didn\'t to have time to consult with all the people I had to consult.',
 '"I could not make a decision in that time, so I told them to do what they wanted."',
 'The 20-time Grand Slam champion has voiced doubts about the wisdom of the one-week format to be introduced by organisers Kosmos, who have promised the International Tennis Federation up to $3 billion in prize money over the next quarter-century.',
 'The competition is set to feature 18 countries in the November 18-24 finals in Madrid next year, and will 

# Text summary using gensim

In [53]:
from gensim.summarization import summarize
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [52]:
r=requests.get('https://click.palletsprojects.com/en/7.x/why')
r.encoding='utf-8'
soup=BeautifulSoup(r.text)
click=soup.find(id='why-click').text.replace('\n',' ')

In [54]:
pd.set_option('display.max_colwidth', 500)

In [56]:
def show_sentences(text):
    return pd.DataFrame({'sentence': sent_tokenize(text)})

In [57]:
click

' Why Click?¶ There are so many libraries out there for writing command line utilities; why does Click exist? This question is easy to answer: because there is not a single command line utility for Python out there which ticks the following boxes:  is lazily composable without restrictions supports implementation of Unix/POSIX command line conventions supports loading values from environment variables out of the box supports for prompting of custom values is fully nestable and composable works the same in Python 2 and 3 supports file handling out of the box comes with useful common helpers (getting terminal dimensions, ANSI colors, fetching direct keyboard input, screen clearing, finding config paths, launching apps and editors, etc.)  There are many alternatives to Click and you can have a look at them if you enjoy them better.  The obvious ones are optparse and argparse from the standard library. Click actually implements its own parsing of arguments and does not use optparse or argp

Note that there are lost the formatting of the originak article since we extracted text from web. We use the show_sentences() func to show sentences.

In [60]:
click_df=show_sentences(click)

In [61]:
click_df.head()

,sentence
0,Why Click?¶ There are so many libraries out there for writing command line utilities; why does Click exist?
1,This question is easy to answer: because there is not a single command line utility for Python out there which ticks the following boxes: is lazily composable without restrictions supports implementation of Unix/POSIX command line conventions supports loading values from environment variables out of the box supports for prompting of custom values is fully nestable and composable works the same in Python 2 and 3 supports file handling out of the box comes with useful common helpers (getting ...
2,There are many alternatives to Click and you can have a look at them if you enjoy them better.
3,The obvious ones are optparse and argparse from the standard library.
4,Click actually implements its own parsing of arguments and does not use optparse or argparse following the optparse parsing behavior.


In [63]:
summary=summarize(click)

In [64]:
summary

'This question is easy to answer: because there is not a single command line utility for Python out there which ticks the following boxes:  is lazily composable without restrictions supports implementation of Unix/POSIX command line conventions supports loading values from environment variables out of the box supports for prompting of custom values is fully nestable and composable works the same in Python 2 and 3 supports file handling out of the box comes with useful common helpers (getting terminal dimensions, ANSI colors, fetching direct keyboard input, screen clearing, finding config paths, launching apps and editors, etc.)  There are many alternatives to Click and you can have a look at them if you enjoy them better.\nClick actually implements its own parsing of arguments and does not use optparse or argparse following the optparse parsing behavior.\nThe reason it’s not based on argparse is that argparse does not allow proper nesting of commands by design and has some deficiencies

# Summarizing Text Using Word Frequency
One of the simple ways to do text summarization is to compute word frquency and extract sentences that contain the words that are most common in the text. Below are processings:
* Ignore stop words
* Determione top words
* Select top words
* Select top sentences


In [14]:
from collections import Counter
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from heapq import nlargest

In [15]:
stopword_punc=set(stopwords.words('english') + list(punctuation))
min_word_prop, max_word_prop=0.1, 0.9


Compute the word frequencies

In [26]:
def word_freqencies(ws):
    words=[word for sentence in ws for word in sentence if word not in stopword_punc]
    counter=Counter(words)
    limit=float(max(counter.values()))
    word_fq={word: freq/limit for word, freq in counter.items()}
# Drop words if too common or too uncommon
    word_fq = {word: freq for word, freq in word_fq.items() if freq > min_word_prop and freq > max_word_prop}
    return word_fq

# score each sentence by how frequent the words
def sentence_score(word_sent, word_fq):
    return sum([word_fq.get(word,0) for word in word_sent])

In [27]:
def summarize(text:str, num_sents=3):
    """
    summarize the test, by return the most relevant sentences
    :text the text to summarize
    : num_sentence the number of sentences to return
    """
    #Make text lower
    text=text.lower()
    # break text into sentences
    sentences=sent_tokenize(text)
    # break sentences into words
    words=[word_tokenize(sentence) for sentence in sentences]
    # Compute the word frequencies
    word_freq=word_freqencies(words)
    # Calculate scores for each sentences
    scores=[sentence_score(word_sentence, word_freq) for word_sentence in words]
    sentence_scores=list(zip(sentences, scores))
    # Rank the sentences
    top_sent_scores=nlargest(num_sents, sentence_scores,key=lambda t: t[1])
    
    return [t[0] for t in top_sent_scores]

In [12]:
with open('data_ch6/PolarVortex.txt','r') as f:
    vortex=f.read()

In [8]:
vortex

'On the coldest day in two decades on his fifth-generation dairy farm, Chris Pollack grabbed a thick black hose from the barn and ventured into the subzero cold,\nwhere his beef cattle were chomping cud and waiting for water.\nThe power had briefly gone out the previous morning, long enough to freeze the line that automatically fills the animalsâ€™ heated water trough. Pollack was here to replace it.\n\n"Are you serious?" Pollack said, peering inside the black hose. "Thereâ€™s water frozen in the end already."\nHe lifted it up to a small space heater and waited for it to thaw.\nSuch is life in the Deep Freeze of 2019.\nThe past 48 hours in the American Midwest have been about endurance, as a breathtaking cold settled in over a massive stretch of the country. \nThe record-setting frigid temperatures, some of the coldest on the planet Thursday, have frozen the Great Lakes, taxed electrical and natural gas infrastructure,\n endangered livestock and tested the mettle of millions who are us

In [9]:
len(sent_tokenize(vortex))

12

In [28]:
summarize(vortex)

['on the coldest day in two decades on his fifth-generation dairy farm, chris pollack grabbed a thick black hose from the barn and ventured into the subzero cold,\nwhere his beef cattle were chomping cud and waiting for water.',
 'the record-setting frigid temperatures, some of the coldest on the planet thursday, have frozen the great lakes, taxed electrical and natural gas infrastructure,\n endangered livestock and tested the mettle of millions who are used to the cold but had never experienced anything like this.',
 '"thereâ€™s water frozen in the end already."']

In [29]:
summarize(vortex, num_sents=1)

['on the coldest day in two decades on his fifth-generation dairy farm, chris pollack grabbed a thick black hose from the barn and ventured into the subzero cold,\nwhere his beef cattle were chomping cud and waiting for water.']